### Detour data

In [ ]:
### Marking
### Go through and mark if they are ready or not - skips ones that are already marked as ready

import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib qt

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

                
if os.path.exists("detour/Still_dupes.csv"):
    df_dupes = pd.read_csv("detour/Still_dupes.csv")
elif os.path.exists("detour/Dupes_in_boxes.csv"):
    df_dupes = pd.read_csv("detour/Dupes_in_boxes.csv")
else:
    df_dupes = pd.DataFrame(columns=["Video","Frame","Box"])
dupe_list = pd.unique(df_dupes.Video)

if os.path.exists("detour/Detour_data_new.csv"):
    df = pd.read_csv("detour/Detour_data_new.csv")
    done_df = df[df.Done == "Yes"]
    done_vids = pd.unique(done_df.Video)
    not_done = pd.unique(df.Video[df.Done == "No"])
else:
    df = pd.read_csv("detour/Detour_data.csv")
    temp = ["No"] * len(df)
    df["Done"] = temp
    done_vids = []
    not_done = pd.unique(df.Video)
df[["x_coord", "y_coord"]] = df[["x_coord", "y_coord"]].apply(pd.to_numeric)
df_vids = pd.unique(df.Video)

filepath_list = list(clear_filepath("detour/track_images_clean"))

path_list = []
img_list = []
for filepath in filepath_list:
    img_name = os.path.basename(filepath)
    vid_name = img_name.replace("_track.jpg", ".mp4")
    if vid_name not in done_vids:
        path_list.append(filepath)
        img_list.append(vid_name)

for vid in not_done:
    if vid not in img_list:
        temp = df[df.Video == vid]
        done_df = pd.concat([done_df, temp], ignore_index = True)
        print("Note:", vid, "lacks track image")

print("Number of videos to go through:", len(path_list))
print(path_list)

count = 0

class plotter:
    def __init__(self, filepath):
        print("1")
        global df
        self.df_all = df.copy()
        global df_dupes
        dupe_list = list(pd.unique(df_dupes.Video))
        self.plotting(filepath[count])
    
    def press(self, event):
        global count
        if event.key == "y":
            count += 1
            index_vid = self.df_all[self.df_all.Video == self.vid_name].index
            self.df_all.loc[index_vid,"Done"] = "Yes"
            try:
                print(count + 1)
                self.plotting(path_list[count])
            except IndexError:
                plt.close()
                self.df_all.to_csv("detour/Detour_data_new.csv", index=False)
                print("Done")
        
        if event.key == "n":
            count += 1
            try:
                print(count+1)
                self.plotting(path_list[count])
            except IndexError:
                plt.close()
                self.df_all.to_csv("detour/Detour_data_new.csv", index=False)
                print("Done")
    
    def plotting(self, filepath):
        plt.close()
        self.df = pd.DataFrame(columns=["Video","Frame","Box","Fish","x_coord","y_coord","Past_barrier","Certainty"])
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot()
        file_name = os.path.basename(filepath)
        print(file_name)
        file_name = file_name.replace("_track.jpg", "")
        self.vid_name = file_name+".mp4"
        self.vid_data = self.df_all[self.df_all.Video == self.vid_name]
        
        x=self.vid_data["x_coord"].values.tolist()
        y=self.vid_data["y_coord"].values.tolist()
        x1=[]
        y1=[]
        x2=[]
        y2=[]
        x3=[]
        y3=[]
        x4=[]
        y4=[]
        x_firsts=[]
        y_firsts=[]
        for box in pd.unique(self.vid_data.Box):
            temp_df = self.vid_data[self.vid_data.Box == box]
            if box == 1:
                x1=temp_df["x_coord"].values.tolist()
                y1=temp_df["y_coord"].values.tolist()
                x_firsts.append(x1[0])
                y_firsts.append(y1[0])
            elif box == 2:
                x2=temp_df["x_coord"].values.tolist()
                y2=temp_df["y_coord"].values.tolist()
                x_firsts.append(x2[0])
                y_firsts.append(y2[0])
            elif box == 3:
                x3=temp_df["x_coord"].values.tolist()
                y3=temp_df["y_coord"].values.tolist()
                x_firsts.append(x3[0])
                y_firsts.append(y3[0])
            elif box == 4:
                x4=temp_df["x_coord"].values.tolist()
                y4=temp_df["y_coord"].values.tolist()
                x_firsts.append(x4[0])
                y_firsts.append(y4[0])
        
        img = plt.imread(filepath)
        if len(x1) > 0:
            self.test1 = plt.plot(x1,y1,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x2) > 0:
            self.test2 = plt.plot(x2,y2,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x3) > 0:
            self.test3 = plt.plot(x3,y3,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x4) > 0:
            self.test4 = plt.plot(x4,y4,linewidth=0.5,color="yellow",alpha=0.5)
        self.sc = plt.scatter(x,y,c="yellow", s=5, alpha=0.2)
        self.sc_first = plt.scatter(x_firsts,y_firsts,c="lime", s=5, alpha=0.5)
        
        global dupe_list
        if self.vid_name in dupe_list:
            xd = []
            yd = []
            dupes = df_dupes[df_dupes.Video == self.vid_name]
            for frame in pd.unique(dupes.Frame):
                dupe_frame_df = dupes[dupes.Frame == frame]
                for box in pd.unique(dupe_frame_df.Box):
                    temp = self.vid_data[(self.vid_data.Frame == frame) & (self.vid_data.Box == box)]
                    tempx = temp["x_coord"].values.tolist()
                    for i in tempx:
                        xd.append(i)
                    tempy = temp["y_coord"].values.tolist()
                    for i in tempy:
                        yd.append(i)
            self.sc_dupes = plt.scatter(xd,yd,c="red", s=5, alpha=0.2)
        self.ax.imshow(img)
        
        plt.show()
        self.fig.canvas.mpl_connect("key_press_event", self.press)


if len(path_list) > 0:
    Plot = plotter(path_list)
else:
    print("No qualifying tracks")

In [ ]:
### Selecting
### Use the graphs to choose the false detection points - press "n" to go to the next image and save
# Continue and skip the ones already done if there are any

import os
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib qt

df_all = pd.read_csv("detour/Detour_data_new.csv")
list_done = pd.unique(df_all.Video[df_all.Done == "Yes"])

if os.path.exists("detour/Still_dupes.csv"):
    df_dupes = pd.read_csv("detour/Still_dupes.csv")
elif os.path.exists("detour/Dupes_in_boxes.csv"):
    df_dupes = pd.read_csv("detour/Dupes_in_boxes.csv")
else:
    df_dupes = pd.DataFrame(columns=["Video","Frame","Box"])
dupe_list = pd.unique(df_dupes.Video)

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

img_list_all = list(clear_filepath("detour/track_images_clean"))
img_list = img_list_all.copy()
for img_dir in img_list_all:
    img = os.path.basename(img_dir)
    vid_name = img.removesuffix("_track.jpg") + ".mp4"
    if vid_name in list_done:
        img_list.remove(img_dir)

print("To do:", len(img_list))

count = 0

class plotter:
    def __init__(self, filepath):
        print("1")
        global df_all
        self.plotting(filepath[count])
    
    def press(self, event):
        global count
        if event.key == "n":
            count += 1
            if len(self.df) > 0:
                if os.path.exists("detour/To_remove.csv"):
                    self.df.to_csv("detour/To_remove.csv", index=False, header=False, mode="a")
                else:
                    self.df.to_csv("detour/To_remove.csv", index=False)
            try:
                print(count + 1)
                self.plotting(img_list[count])
            except IndexError:
                plt.close()
                print("Done")
    
    def hover(self, event):
        vis = self.annot.get_visible()
        if event.inaxes == self.ax:
            cont, ind = self.sc.contains(event)
            if cont:
                self.update_annot(ind)
                self.annot.set_visible(True)
                self.fig.canvas.draw_idle()
            else:
                if vis:
                    self.annot.set_visible(False)
                    self.fig.canvas.draw_idle()
    
    def on_click(self, event):
        if event.inaxes == self.ax:
            cont, ind = self.sc.contains(event)
            if cont:
                for n in ind["ind"]:
                    temp_df = pd.DataFrame({"Video": [self.vid_name], "Frame": [self.frame[n]], "ID": [self.fish_id[n]]})
                    self.df = pd.concat([self.df, temp_df], ignore_index=True)
                
    
    def update_annot(self, ind):
    
        pos = self.sc.get_offsets()[ind["ind"][0]]
        self.annot.xy = pos
        text = "{}, {}".format(" ".join([str(self.frame[n]) for n in ind["ind"]]), 
                               " ".join([str(self.fish_id[n]) for n in ind["ind"]]))
        self.annot.set_text(text)
        self.annot.get_bbox_patch().set_facecolor("yellow")
        self.annot.get_bbox_patch().set_alpha(0.4)
    
    def plotting(self, filepath):
        plt.close()
        self.df = pd.DataFrame(columns=["Video", "Frame", "ID"])
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot()
        file_name = os.path.basename(filepath)
        print(file_name)
        file_name = file_name.replace("_track.jpg", "")
        self.vid_name = file_name + ".mp4"
        vid_data = []
        self.vid_data = df_all[df_all.Video == self.vid_name]
        
        self.frame = self.vid_data["Frame"].values.tolist()
        self.fish_id = self.vid_data["Fish"].values.tolist()
        
        x=self.vid_data["x_coord"].values.tolist()
        y=self.vid_data["y_coord"].values.tolist()
        x1=[]
        y1=[]
        x2=[]
        y2=[]
        x3=[]
        y3=[]
        x4=[]
        y4=[]
        x_firsts=[]
        y_firsts=[]
        for box in pd.unique(self.vid_data.Box):
            temp_df = self.vid_data[self.vid_data.Box == box]
            if box == 1:
                x1=temp_df["x_coord"].values.tolist()
                y1=temp_df["y_coord"].values.tolist()
                x_firsts.append(x1[0])
                y_firsts.append(y1[0])
            elif box == 2:
                x2=temp_df["x_coord"].values.tolist()
                y2=temp_df["y_coord"].values.tolist()
                x_firsts.append(x2[0])
                y_firsts.append(y2[0])
            elif box == 3:
                x3=temp_df["x_coord"].values.tolist()
                y3=temp_df["y_coord"].values.tolist()
                x_firsts.append(x3[0])
                y_firsts.append(y3[0])
            elif box == 4:
                x4=temp_df["x_coord"].values.tolist()
                y4=temp_df["y_coord"].values.tolist()
                x_firsts.append(x4[0])
                y_firsts.append(y4[0])
        
        img = plt.imread(filepath)
        if len(x1) > 0:
            self.test1 = plt.plot(x1,y1,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x2) > 0:
            self.test2 = plt.plot(x2,y2,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x3) > 0:
            self.test3 = plt.plot(x3,y3,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x4) > 0:
            self.test4 = plt.plot(x4,y4,linewidth=0.5,color="yellow",alpha=0.5)
        self.sc = plt.scatter(x,y,c="yellow", s=5, alpha=0.2)
        self.sc_first = plt.scatter(x_firsts,y_firsts,c="lime", s=5, alpha=0.5)
        
        global dupe_list
        if self.vid_name in dupe_list:
            xd = []
            yd = []
            dupes = df_dupes[df_dupes.Video == self.vid_name]
            for frame in pd.unique(dupes.Frame):
                dupe_frame_df = dupes[dupes.Frame == frame]
                for box in pd.unique(dupe_frame_df.Box):
                    temp = self.vid_data[(self.vid_data.Frame == frame) & (self.vid_data.Box == box)]
                    tempx = temp["x_coord"].values.tolist()
                    for i in tempx:
                        xd.append(i)
                    tempy = temp["y_coord"].values.tolist()
                    for i in tempy:
                        yd.append(i)
            self.sc_dupes = plt.scatter(xd,yd,c="red", s=5, alpha=0.2)
        
        self.ax.imshow(img)

        self.annot = self.ax.annotate("", xy=(0,0), xytext=(20,20), textcoords="offset points", 
                        bbox=dict(boxstyle="round", fc="w"), 
                        arrowprops=dict(arrowstyle="->"))
        self.annot.set_visible(False)
        
        plt.show()
        self.fig.canvas.mpl_connect("key_press_event", self.press)
        self.fig.canvas.mpl_connect("motion_notify_event", self.hover)
        self.fig.canvas.mpl_connect("button_press_event", self.on_click)

Plot = plotter(img_list)

In [ ]:
### Clearing
### Clear out the incorrect ID numbers, save new tracks, and create a new still dupe file

import os
import pandas as pd
import cv2
import shutil
import numpy as np

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

coord_file = pd.read_csv("detour/Detour_data_new.csv")

# create a backup - if already exist, give it a new number
backup_path = "detour/backups"
backup_list = list(clear_filepath(backup_path))
backup_nr = str(len(backup_list)+1)
new_file_dir = "detour/backups/Detour_data_new-backup" + backup_nr + ".csv"
coord_file.to_csv(new_file_dir, index=False)
    
# import the csv with the ones to remove
remove_ids = pd.read_csv("detour/To_remove.csv")

# remove the IDs
cleared_coords = pd.DataFrame(columns=coord_file.columns)

for vid_name in pd.unique(coord_file.Video):
    vid_removes = remove_ids[remove_ids.Video == vid_name]
    temp_coords = coord_file[coord_file.Video == vid_name]
    frame_fish = vid_removes[["Frame", "ID"]].values.tolist()
    for frame, fish_id in frame_fish:
        index_nr = temp_coords.loc[(temp_coords.Frame == frame) & (temp_coords.Fish == fish_id)].index
        temp_coords = temp_coords.drop(index_nr)
    cleared_coords = pd.concat([cleared_coords, temp_coords], ignore_index = True)
    
# save it as a new one (with same name as old - the old one should now have been saved as backup)
cleared_coords.to_csv("detour/Detour_data_new.csv", index=False)

# create new track with the images behind 
clear_img_dir = "detour/track_images_clean"
clear_img_list = list(clear_filepath(clear_img_dir))
for vid_name in pd.unique(remove_ids.Video):
    vid_df = cleared_coords[cleared_coords.Video == vid_name]
    img_name = vid_name.removesuffix(".mp4") + "_track.jpg"
    img_dest_new = "detour/track_images/" + img_name
    current_image = cv2.imread(clear_img_dir + "/" + img_name)
    for box_id in pd.unique(vid_df.Box):
        vid_df_temp = vid_df[vid_df.Box == box_id]
        coord_list = vid_df_temp[["x_coord", "y_coord"]].values.tolist()
        pts = np.array(coord_list, np.int32)
        cv2.polylines(current_image,[pts],False,(0,255,255))
    cv2.imwrite(img_dest_new,current_image)

# Create a new file of dupes
dupes_df = pd.DataFrame(columns = ["Video", "Frame", "Box"])

for vid in pd.unique(coord_file.Video):
    vid_df = coord_file[coord_file.Video == vid]
    if pd.unique(vid_df.Done) == "Yes":
        continue
    dupes = False
    dupes_vid = pd.DataFrame(columns = ["Video", "Frame", "Box"])
    for frame in pd.unique(vid_df.Frame):
        frame_df = vid_df[vid_df.Frame == frame]
        for box in pd.unique(frame_df.Box):
            if len(pd.unique(frame_df.Fish[frame_df.Box == box])) > 1:
                dupes = True
                temp_df = pd.DataFrame([[vid, frame, box]], columns = ["Video", "Frame", "Box"])
                dupes_vid = pd.concat([dupes_vid, temp_df], ignore_index = True)
    
    if dupes == True:
        print(vid, "still has dupes")
        dupes_df = pd.concat([dupes_df, dupes_vid], ignore_index = True)

dupes_df.to_csv("detour/Still_dupes.csv", index=False) 

print("Done")

### Open field and mirror data

In [ ]:
### Marking
### Go through and remark if they are ready or not - skips ones that are already marked as ready

import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib qt

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

                
if os.path.exists("OF_mirror/Still_dupes.csv"):
    df_dupes = pd.read_csv("OF_mirror/Still_dupes.csv")
elif os.path.exists("OF_mirror/Dupes_in_boxes.csv"):
    df_dupes = pd.read_csv("OF_mirror/Dupes_in_boxes.csv")
else:
    df_dupes = pd.DataFrame(columns=["Video","Frame","Box"])
dupe_list = pd.unique(df_dupes.Video)

if os.path.exists("OF_mirror/OF_mirror_data_new.csv"):
    df = pd.read_csv("OF_mirror/OF_mirror_data_new.csv")
    done_df = df[df.Done == "Yes"]
    done_vids = pd.unique(done_df.Video)
    not_done = pd.unique(df.Video[df.Done == "No"])
else:
    df = pd.read_csv("OF_mirror/OF_mirror_data.csv")
    temp = ["No"] * len(df)
    df["Done"] = temp
    done_vids = []
    not_done = pd.unique(df.Video)
df[["x_coord", "y_coord"]] = df[["x_coord", "y_coord"]].apply(pd.to_numeric)
df_vids = pd.unique(df.Video)

filepath_list = list(clear_filepath("OF_mirror/track_images_clean"))

path_list = []
img_list = []
for filepath in filepath_list:
    img_name = os.path.basename(filepath)
    vid_name = img_name.replace("_track.jpg", ".mp4")
    if vid_name not in done_vids:
        path_list.append(filepath)
        img_list.append(vid_name)

for vid in not_done:
    if vid not in img_list:
        temp = df[df.Video == vid]
        done_df = pd.concat([done_df, temp], ignore_index = True)
        print("Note:", vid, "lacks track image")

print("Number of videos to go through:", len(path_list))
print(path_list)

count = 0

class plotter:
    def __init__(self, filepath):
        print("1")
        global df
        self.df_all = df.copy()
        global df_dupes
        dupe_list = list(pd.unique(df_dupes.Video))
        self.plotting(filepath[count])
    
    def press(self, event):
        global count
        if event.key == "y":
            count += 1
            index_vid = self.df_all[self.df_all.Video == self.vid_name].index
            self.df_all.loc[index_vid,"Done"] = "Yes"
            try:
                print(count + 1)
                self.plotting(path_list[count])
            except IndexError:
                plt.close()
                self.df_all.to_csv("OF_mirror/OF_mirror_data_new.csv", index=False)
                print("Done")
        
        if event.key == "n":
            count += 1
            try:
                print(count+1)
                self.plotting(path_list[count])
            except IndexError:
                plt.close()
                self.df_all.to_csv("OF_mirror/OF_mirror_data_new.csv", index=False)
                print("Done")
    
    def plotting(self, filepath):
        plt.close()
        self.df = pd.DataFrame(columns=["Video","Frame","Box","Fish","x_coord","y_coord","Past_barrier","Certainty"])
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot()
        file_name = os.path.basename(filepath)
        print(file_name)
        file_name = file_name.replace("_track.jpg", "")
        self.vid_name = file_name+".mp4"
        self.vid_data = self.df_all[self.df_all.Video == self.vid_name]
        
        x=self.vid_data["x_coord"].values.tolist()
        y=self.vid_data["y_coord"].values.tolist()
        x1=[]
        y1=[]
        x2=[]
        y2=[]
        x3=[]
        y3=[]
        x4=[]
        y4=[]
        x_firsts=[]
        y_firsts=[]
        for box in pd.unique(self.vid_data.Box):
            temp_df = self.vid_data[self.vid_data.Box == box]
            if box == 1:
                x1=temp_df["x_coord"].values.tolist()
                y1=temp_df["y_coord"].values.tolist()
                x_firsts.append(x1[0])
                y_firsts.append(y1[0])
            elif box == 2:
                x2=temp_df["x_coord"].values.tolist()
                y2=temp_df["y_coord"].values.tolist()
                x_firsts.append(x2[0])
                y_firsts.append(y2[0])
            elif box == 3:
                x3=temp_df["x_coord"].values.tolist()
                y3=temp_df["y_coord"].values.tolist()
                x_firsts.append(x3[0])
                y_firsts.append(y3[0])
            elif box == 4:
                x4=temp_df["x_coord"].values.tolist()
                y4=temp_df["y_coord"].values.tolist()
                x_firsts.append(x4[0])
                y_firsts.append(y4[0])
        
        img = plt.imread(filepath)
        if len(x1) > 0:
            self.test1 = plt.plot(x1,y1,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x2) > 0:
            self.test2 = plt.plot(x2,y2,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x3) > 0:
            self.test3 = plt.plot(x3,y3,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x4) > 0:
            self.test4 = plt.plot(x4,y4,linewidth=0.5,color="yellow",alpha=0.5)
        self.sc = plt.scatter(x,y,c="yellow", s=5, alpha=0.2)
        self.sc_first = plt.scatter(x_firsts,y_firsts,c="lime", s=5, alpha=0.5)
        
        global dupe_list
        if self.vid_name in dupe_list:
            xd = []
            yd = []
            dupes = df_dupes[df_dupes.Video == self.vid_name]
            for frame in pd.unique(dupes.Frame):
                dupe_frame_df = dupes[dupes.Frame == frame]
                for box in pd.unique(dupe_frame_df.Box):
                    temp = self.vid_data[(self.vid_data.Frame == frame) & (self.vid_data.Box == box)]
                    tempx = temp["x_coord"].values.tolist()
                    for i in tempx:
                        xd.append(i)
                    tempy = temp["y_coord"].values.tolist()
                    for i in tempy:
                        yd.append(i)
            self.sc_dupes = plt.scatter(xd,yd,c="red", s=5, alpha=0.2)
        self.ax.imshow(img)
        
        plt.show()
        self.fig.canvas.mpl_connect("key_press_event", self.press)


if len(path_list) > 0:
    Plot = plotter(path_list)
else:
    print("No qualifying tracks")

In [ ]:
### Selecting
### Use the graphs to choose the false detection points - press "n" to go to the next image and save
# Continue and skip the ones already done if there are any

import os
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib qt

df_all = pd.read_csv("OF_mirror/OF_mirror_data_new.csv")
list_done = pd.unique(df_all.Video[df_all.Done == "Yes"])

if os.path.exists("OF_mirror/Still_dupes.csv"):
    df_dupes = pd.read_csv("OF_mirror/Still_dupes.csv")
elif os.path.exists("OF_mirror/Dupes_in_boxes.csv"):
    df_dupes = pd.read_csv("OF_mirror/Dupes_in_boxes.csv")
else:
    df_dupes = pd.DataFrame(columns=["Video","Frame","Box"])
dupe_list = pd.unique(df_dupes.Video)

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

img_list_all = list(clear_filepath("OF_mirror/track_images_clean"))
img_list = img_list_all.copy()
for img_dir in img_list_all:
    img = os.path.basename(img_dir)
    vid_name = img.removesuffix("_track.jpg") + ".mp4"
    if vid_name in list_done:
        img_list.remove(img_dir)

print("To do:", len(img_list))

count = 0

class plotter:
    def __init__(self, filepath):
        print("1")
        global df_all
        self.plotting(filepath[count])
    
    def press(self, event):
        global count
        if event.key == "n":
            count += 1
            if len(self.df) > 0:
                if os.path.exists("OF_mirror/To_remove.csv"):
                    self.df.to_csv("OF_mirror/To_remove.csv", index=False, header=False, mode="a")
                else:
                    self.df.to_csv("OF_mirror/To_remove.csv", index=False)
            try:
                print(count + 1)
                self.plotting(img_list[count])
            except IndexError:
                plt.close()
                print("Done")
    
    def hover(self, event):
        vis = self.annot.get_visible()
        if event.inaxes == self.ax:
            cont, ind = self.sc.contains(event)
            if cont:
                self.update_annot(ind)
                self.annot.set_visible(True)
                self.fig.canvas.draw_idle()
            else:
                if vis:
                    self.annot.set_visible(False)
                    self.fig.canvas.draw_idle()
    
    def on_click(self, event):
        if event.inaxes == self.ax:
            cont, ind = self.sc.contains(event)
            if cont:
                for n in ind["ind"]:
                    temp_df = pd.DataFrame({"Video": [self.vid_name], "Frame": [self.frame[n]], "ID": [self.fish_id[n]]})
                    self.df = pd.concat([self.df, temp_df], ignore_index=True)
                
    
    def update_annot(self, ind):
    
        pos = self.sc.get_offsets()[ind["ind"][0]]
        self.annot.xy = pos
        text = "{}, {}".format(" ".join([str(self.frame[n]) for n in ind["ind"]]), 
                               " ".join([str(self.fish_id[n]) for n in ind["ind"]]))
        self.annot.set_text(text)
        self.annot.get_bbox_patch().set_facecolor("yellow")
        self.annot.get_bbox_patch().set_alpha(0.4)
    
    def plotting(self, filepath):
        plt.close()
        self.df = pd.DataFrame(columns=["Video", "Frame", "ID"])
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot()
        file_name = os.path.basename(filepath)
        print(file_name)
        file_name = file_name.replace("_track.jpg", "")
        self.vid_name = file_name + ".mp4"
        vid_data = []
        self.vid_data = df_all[df_all.Video == self.vid_name]
        
        self.frame = self.vid_data["Frame"].values.tolist()
        self.fish_id = self.vid_data["Fish"].values.tolist()
        
        x=self.vid_data["x_coord"].values.tolist()
        y=self.vid_data["y_coord"].values.tolist()
        x1=[]
        y1=[]
        x2=[]
        y2=[]
        x3=[]
        y3=[]
        x4=[]
        y4=[]
        x_firsts=[]
        y_firsts=[]
        for box in pd.unique(self.vid_data.Box):
            temp_df = self.vid_data[self.vid_data.Box == box]
            if box == 1:
                x1=temp_df["x_coord"].values.tolist()
                y1=temp_df["y_coord"].values.tolist()
                x_firsts.append(x1[0])
                y_firsts.append(y1[0])
            elif box == 2:
                x2=temp_df["x_coord"].values.tolist()
                y2=temp_df["y_coord"].values.tolist()
                x_firsts.append(x2[0])
                y_firsts.append(y2[0])
            elif box == 3:
                x3=temp_df["x_coord"].values.tolist()
                y3=temp_df["y_coord"].values.tolist()
                x_firsts.append(x3[0])
                y_firsts.append(y3[0])
            elif box == 4:
                x4=temp_df["x_coord"].values.tolist()
                y4=temp_df["y_coord"].values.tolist()
                x_firsts.append(x4[0])
                y_firsts.append(y4[0])
        
        img = plt.imread(filepath)
        if len(x1) > 0:
            self.test1 = plt.plot(x1,y1,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x2) > 0:
            self.test2 = plt.plot(x2,y2,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x3) > 0:
            self.test3 = plt.plot(x3,y3,linewidth=0.5,color="yellow",alpha=0.5)
        if len(x4) > 0:
            self.test4 = plt.plot(x4,y4,linewidth=0.5,color="yellow",alpha=0.5)
        self.sc = plt.scatter(x,y,c="yellow", s=5, alpha=0.2)
        self.sc_first = plt.scatter(x_firsts,y_firsts,c="lime", s=5, alpha=0.5)
        
        global dupe_list
        if self.vid_name in dupe_list:
            xd = []
            yd = []
            dupes = df_dupes[df_dupes.Video == self.vid_name]
            for frame in pd.unique(dupes.Frame):
                dupe_frame_df = dupes[dupes.Frame == frame]
                for box in pd.unique(dupe_frame_df.Box):
                    temp = self.vid_data[(self.vid_data.Frame == frame) & (self.vid_data.Box == box)]
                    tempx = temp["x_coord"].values.tolist()
                    for i in tempx:
                        xd.append(i)
                    tempy = temp["y_coord"].values.tolist()
                    for i in tempy:
                        yd.append(i)
            self.sc_dupes = plt.scatter(xd,yd,c="red", s=5, alpha=0.2)
        
        self.ax.imshow(img)

        self.annot = self.ax.annotate("", xy=(0,0), xytext=(20,20), textcoords="offset points", 
                        bbox=dict(boxstyle="round", fc="w"), 
                        arrowprops=dict(arrowstyle="->"))
        self.annot.set_visible(False)
        
        plt.show()
        self.fig.canvas.mpl_connect("key_press_event", self.press)
        self.fig.canvas.mpl_connect("motion_notify_event", self.hover)
        self.fig.canvas.mpl_connect("button_press_event", self.on_click)

Plot = plotter(img_list)

In [ ]:
### Clearing
### Clear out the incorrect ID numbers, save new tracks, and create a still dupes file

import os
import pandas as pd
import cv2
import shutil
import numpy as np

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

coord_file = pd.read_csv("OF_mirror/OF_mirror_data_new.csv")

# create a backup - if already exist, give it a new number
backup_path = "OF_mirror/backups"
backup_list = list(clear_filepath(backup_path))
backup_nr = str(len(backup_list)+1)
new_file_dir = "OF_mirror/backups/OF_mirror_data_new-backup" + backup_nr + ".csv"
coord_file.to_csv(new_file_dir, index=False)
    
# import the csv with the ones to remove
remove_ids = pd.read_csv("OF_mirror/To_remove.csv")

# remove the IDs
cleared_coords = pd.DataFrame(columns=coord_file.columns)

for vid_name in pd.unique(coord_file.Video):
    vid_removes = remove_ids[remove_ids.Video == vid_name]
    temp_coords = coord_file[coord_file.Video == vid_name]
    frame_fish = vid_removes[["Frame", "ID"]].values.tolist()
    for frame, fish_id in frame_fish:
        index_nr = temp_coords.loc[(temp_coords.Frame == frame) & (temp_coords.Fish == fish_id)].index
        temp_coords = temp_coords.drop(index_nr)
    cleared_coords = pd.concat([cleared_coords, temp_coords], ignore_index = True)
    
# save it as a new one (with same name as old - the old one should now have been saved as backup)
cleared_coords.to_csv("OF_mirror/OF_mirror_data_new.csv", index=False)

# create new track with the images behind 
clear_img_dir = "OF_mirror/track_images_clean"
clear_img_list = list(clear_filepath(clear_img_dir))
for vid_name in pd.unique(remove_ids.Video):
    vid_df = cleared_coords[cleared_coords.Video == vid_name]
    img_name = vid_name.removesuffix(".mp4") + "_track.jpg"
    img_dest_new = "OF_mirror/track_images/" + img_name
    current_image = cv2.imread(clear_img_dir + "/" + img_name)
    for box_id in pd.unique(vid_df.Box):
        vid_df_temp = vid_df[vid_df.Box == box_id]
        coord_list = vid_df_temp[["x_coord", "y_coord"]].values.tolist()
        pts = np.array(coord_list, np.int32)
        cv2.polylines(current_image,[pts],False,(0,255,255))
    cv2.imwrite(img_dest_new,current_image)

# Create a new file of dupes
dupes_df = pd.DataFrame(columns = ["Video", "Frame", "Box"])

for vid in pd.unique(coord_file.Video):
    vid_df = coord_file[coord_file.Video == vid]
    if pd.unique(vid_df.Done) == "Yes":
        continue
    dupes = False
    dupes_vid = pd.DataFrame(columns = ["Video", "Frame", "Box"])
    for frame in pd.unique(vid_df.Frame):
        frame_df = vid_df[vid_df.Frame == frame]
        for box in pd.unique(frame_df.Box):
            if len(pd.unique(frame_df.Fish[frame_df.Box == box])) > 1:
                dupes = True
                temp_df = pd.DataFrame([[vid, frame, box]], columns = ["Video", "Frame", "Box"])
                dupes_vid = pd.concat([dupes_vid, temp_df], ignore_index = True)
    
    if dupes == True:
        print(vid, "still has dupes")
        dupes_df = pd.concat([dupes_df, dupes_vid], ignore_index = True)

dupes_df.to_csv("OF_mirror/Still_dupes.csv", index=False) 

print("Done")